## Deliverable 2. Create a Customer Travel Destinations Map.

In [27]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bethel,US,41.3712,-73.4140,90.45,46,1,9.22,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,44.26,57,40,12.66,scattered clouds
2,2,Avarua,CK,-21.2078,-159.7750,75.25,60,75,5.75,broken clouds
3,3,Cape Town,ZA,-33.9258,18.4232,72.01,27,0,13.22,clear sky
4,4,Tiksi,RU,71.6872,128.8694,43.83,64,100,1.54,overcast clouds


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Avarua,CK,-21.2078,-159.7750,75.25,60,75,5.75,broken clouds
7,7,Padang,ID,-0.9492,100.3543,75.60,78,78,3.83,broken clouds
9,9,Hilo,US,19.7297,-155.0900,80.33,72,90,13.80,light rain
10,10,Vaini,TO,-21.2000,-175.2000,77.16,78,75,13.80,broken clouds
14,14,Bathsheba,BB,13.2167,-59.5167,81.21,69,20,20.87,few clouds
15,15,Dingle,PH,10.9995,122.6711,78.48,82,100,2.04,overcast clouds
16,16,Albany,US,42.6001,-73.9662,89.17,59,46,5.03,scattered clouds
23,23,Manado,ID,1.4870,124.8455,77.77,94,40,3.51,light rain
29,29,Mahebourg,MU,-20.4081,57.7000,75.49,78,40,9.22,light rain
32,32,Lufilufi,WS,-13.8745,-171.5986,89.38,74,36,5.06,light rain


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                215
City                   215
Country                214
Lat                    215
Lng                    215
Max Temp               215
Humidity               215
Cloudiness             215
Wind Speed             215
Current Description    215
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.count()

City_ID                214
City                   214
Country                214
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Current Description    214
dtype: int64

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,
7,Padang,ID,75.60,broken clouds,-0.9492,100.3543,
9,Hilo,US,80.33,light rain,19.7297,-155.0900,
10,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,
14,Bathsheba,BB,81.21,few clouds,13.2167,-59.5167,
15,Dingle,PH,78.48,overcast clouds,10.9995,122.6711,
16,Albany,US,89.17,scattered clouds,42.6001,-73.9662,
23,Manado,ID,77.77,light rain,1.4870,124.8455,
29,Mahebourg,MU,75.49,light rain,-20.4081,57.7000,
32,Lufilufi,WS,89.38,light rain,-13.8745,-171.5986,


In [17]:
test_df=hotel_df.copy()
drop_test_df=test_df.dropna()
drop_test_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,
7,Padang,ID,75.60,broken clouds,-0.9492,100.3543,
9,Hilo,US,80.33,light rain,19.7297,-155.0900,
10,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,
14,Bathsheba,BB,81.21,few clouds,13.2167,-59.5167,
15,Dingle,PH,78.48,overcast clouds,10.9995,122.6711,
16,Albany,US,89.17,scattered clouds,42.6001,-73.9662,
23,Manado,ID,77.77,light rain,1.4870,124.8455,
29,Mahebourg,MU,75.49,light rain,-20.4081,57.7000,
32,Lufilufi,WS,89.38,light rain,-13.8745,-171.5986,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
clean_hotel_df=hotel_df.replace("",np.NaN)
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,Paradise Inn
7,Padang,ID,75.60,broken clouds,-0.9492,100.3543,Mercure Padang
9,Hilo,US,80.33,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
10,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
14,Bathsheba,BB,81.21,few clouds,13.2167,-59.5167,Atlantis Hotel
15,Dingle,PH,78.48,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
16,Albany,US,89.17,scattered clouds,42.6001,-73.9662,NaN
23,Manado,ID,77.77,light rain,1.4870,124.8455,Aryaduta Hotel
29,Mahebourg,MU,75.49,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
32,Lufilufi,WS,89.38,light rain,-13.8745,-171.5986,Leuaina Beach Resort and Spa


In [30]:
clean_hotel_df=clean_hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,Paradise Inn
7,Padang,ID,75.60,broken clouds,-0.9492,100.3543,Mercure Padang
9,Hilo,US,80.33,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
10,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
14,Bathsheba,BB,81.21,few clouds,13.2167,-59.5167,Atlantis Hotel
15,Dingle,PH,78.48,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
23,Manado,ID,77.77,light rain,1.4870,124.8455,Aryaduta Hotel
29,Mahebourg,MU,75.49,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
32,Lufilufi,WS,89.38,light rain,-13.8745,-171.5986,Leuaina Beach Resort and Spa
35,Buchanan,LR,76.96,overcast clouds,5.8808,-10.0467,Kamaneahn Hotel Inc


In [39]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [45]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, dissipating=False,
             point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))